<a href="https://colab.research.google.com/github/lishura/Apache_Spark/blob/main/HW5_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

— Используя Spark прочитайте данные из файла csv.
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3c4ec1757904f97bf306d44c19e27008a2cb91dfc166d1aa4a731fdb19312bdc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

In [13]:
spark = SparkSession.builder.appName('Books').getOrCreate()
# csv_file_path = r"C:\Users\User\Documents\обучение GB\Spark\books.csv"
df = spark.read.csv('books.csv', header=True, inferSchema=True)
print("Исходный файл:")
df.show()

Исходный файл:
+--------------------+--------------------+------------------+------+------+
|               title|              author|             genre| sales|  year|
+--------------------+--------------------+------------------+------+------+
|                1984|     "George Orwell"| "Science Fiction"|5000.0|1949.0|
|The Lord of the R...|    "J.R.R. Tolkien"|         "Fantasy"|3000.0|1954.0|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"|4000.0|1960.0|
|The Catcher in th...|     "J.D. Salinger"|           "Novel"|2000.0|1951.0|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"|4500.0|1925.0|
+--------------------+--------------------+------------------+------+------+



In [14]:
filtered_df = df.filter(df.sales > 3000)
filtered_df.show()

+--------------------+--------------------+------------------+------+------+
|               title|              author|             genre| sales|  year|
+--------------------+--------------------+------------------+------+------+
|                1984|     "George Orwell"| "Science Fiction"|5000.0|1949.0|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"|4000.0|1960.0|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"|4500.0|1925.0|
+--------------------+--------------------+------------------+------+------+



In [15]:
window_spec = Window.partitionBy('genre')
sum_sales_by_genre = fn.sum('sales').over(window_spec)
sum_sales_by_genre_df = df.withColumn('sum_sales_by_genre', sum_sales_by_genre)
print('Данные по общему объему продаж в порядке убывания в разрезе жанров:')
sum_sales_by_genre_df.select(['genre', 'sum_sales_by_genre']).distinct().orderBy(sum_sales_by_genre_df.sum_sales_by_genre.desc()).show()

Данные по общему объему продаж в порядке убывания в разрезе жанров:
+------------------+------------------+
|             genre|sum_sales_by_genre|
+------------------+------------------+
|           "Novel"|            6500.0|
| "Science Fiction"|            5000.0|
| "Southern Gothic"|            4000.0|
|         "Fantasy"|            3000.0|
+------------------+------------------+

